# Imports

In [2]:
#####################################################
################## PACKAGES #########################
#####################################################
import pandas as pd
import sys 
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
from sklearn.metrics import accuracy_score, roc_auc_score
from datetime import datetime, timedelta, date
from itertools import combinations
from numpy.linalg import norm
import pickle
import os
import torch.nn as nn
import torch
from torch.nn import Linear
import torch.nn.functional as F
import altair as alt
from altair import expr, datum
from vega_datasets import data
from geopy.geocoders import Nominatim
from tqdm import tqdm
from utils import *

tqdm.pandas()
DATA_PATH = './data'

# First infos

In [3]:
metadata = preprocess_metadata(DATA_PATH)

In [5]:
metadata

,date,id_coord,plume,lat,lon,coord_x,coord_y,path,location,country,city
0,2023-02-23,id_6675,yes,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...,"(Shah Jamal Road, Zaildar Park, Shah Jamal, لا...",پاکستان,لاہور
1,2023-01-03,id_2542,yes,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...,"(沁水县, 晋城市, 山西省, 中国, (35.537987721577416, 112.5...",中国,晋城市
2,2023-03-01,id_6546,yes,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...,"(Chhendipada, Anugul, Odisha, 759130, India, (...",India,None
3,2023-02-25,id_6084,yes,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...,"(Vrindavan, Usar Barauli, Sarojani Nagar, Luck...",India,None
4,2023-01-05,id_2012,yes,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...,"(وادي الحسينية, ناحية هجين, منطقة البوكمال, مح...",سوريا,None
...,...,...,...,...,...,...,...,...,...,...,...
425,2023-03-02,id_6658,no,30.965619,34.541283,39,36,images/no_plume/20230302_methane_mixing_ratio_...,"(מועצה אזורית רמת נגב, נפת באר שבע, מחוז הדרום...",ישראל,מועצה אזורית רמת נגב
426,2023-02-18,id_4690,no,35.950275,40.267652,29,28,images/no_plume/20230218_methane_mixing_ratio_...,"(ناحية الکسرة, منطقة دير الزور, محافظة دير الز...",سوريا,None
427,2023-02-13,id_2519,no,49.963801,6.016938,23,10,images/no_plume/20230213_methane_mixing_ratio_...,"(Schüttbuerger Millen, Alscheid, Kiischpelt, C...",Lëtzebuerg,Kiischpelt
428,2023-02-13,id_5510,no,32.713854,44.609398,55,54,images/no_plume/20230213_methane_mixing_ratio_...,"(قرية ١٧ تموز, ناحية الامام, قضاء المحاويل, مح...",العراق,None


In [6]:
## The geographic distribution
countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    "equirectangular"
).properties(
    width=500,
    height=300
)

points = alt.Chart(metadata).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.value(10),
    tooltip='city'
)

background+points

TypeError: Object of type Location is not JSON serializable

alt.LayerChart(...)

In [67]:
# COUNTRY DISTRIBUTION
country_distrib = metadata.groupby('country').agg(count=('date', 'count')).reset_index()
country_distrib = country_distrib.sort_values(by='count', ascending=False)
hist_country = alt.Chart(country_distrib.sort_values(by='count')).mark_bar().encode(
    x='country:N',
    y=alt.Y('count:Q')
)

hist_country.properties(title='Distribution of the data')

alt.Chart(...)

In [72]:
#proportion of true and false
metadata.plume.value_counts()

yes    216
no     214
Name: plume, dtype: int64